图像分类演示

作者：谢文伟

邮件：jim.xie.cn@outlook.com

主页：https://github.com/jim-xie-cn/ai-cv

# 导入用到的包

In [ ]:
from mlxtend.data import loadlocal_mnist
from skimage import feature
import numpy as np
import cv2
from matplotlib import pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #使用中文字体

# 样本读取和预处理

定义从数据集中读取样本的函数，只保留标签为0或1的图像

In [ ]:
def load_sample(img_file,label_file):
    #从MNIST文件中提取图像和标签数据
    raw_image,raw_label=loadlocal_mnist(images_path=img_file,labels_path=label_file)
    #MNIST数据集包含0～9十个数字分类，这里只使用0和1两个分类
    #将样本和标签合并到一个numpy数组里，根据标签过滤，只保留0和1两个数字的数据
    raw_label = raw_label.reshape(-1,1)
    raw_sample = np.hstack((raw_image,raw_label))
    raw_sample.astype(raw_image.dtype)
    filter=np.where(raw_sample[:,-1]<=1)  #只保留0和1两个数字的数据
    images = raw_sample[filter][:,:-1]
    labels = raw_sample[filter][:,-1]
    #MNIST原始图像格式为(28，28),这里转换为灰度图格式(28,28,1)
    images = images.reshape(images.shape[0],28,28,1)
    return images,labels

读取样本，得到训练集和测试集

In [ ]:
train_image,train_y = load_sample('./dataset/mnist/train-images-idx3-ubyte',
                               './dataset/mnist/train-labels-idx1-ubyte')
test_image,test_y = load_sample('./dataset/mnist/t10k-images-idx3-ubyte',
                              './dataset/mnist/t10k-labels-idx1-ubyte')

#将数据集的图像保存为本地文件

In [ ]:
def save_sample(img_file,label_file):
    images,labels = load_sample(img_file,label_file)
    features = extract_feature(images)
    count = 0
    for x1,y1 in zip(images,labels):
        path = "./%d/"%y1
        try:
            os.mkdir(path)
        except:
            pass
        f = "%s%s.jpg"%(path,count)
        count = count + 1
        print("Saving ",f,x1.shape)
        cv2.imwrite(f,x1)

显示预处理后的样本

In [ ]:
fig,ax = plt.subplots(5,7,figsize=(12,8))
def ShowImg(id,title,img,cm = 'gray'):
    x,y = divmod(id,7)
    ax[x,y].imshow(img,plt.get_cmap(cm))
    ax[x,y].axis('off')
count=0
for label,image in zip(train_y,train_image):
    if count >= 5*7:
        break
    ShowImg(count,label,image.reshape(28,28))
    count = count+1

# 提取图像特征
这里使用HOG特征

In [ ]:
#提取训练图像的HOG特征
train_x = [feature.hog(x) for x in train_image]
train_x = np.array(train_x)
#提取测试图像的HOG特征
test_x = [feature.hog(x) for x in test_image]
test_x = np.array(test_x)

# 选择分类器

导入sklearn的分类模型

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

定义组合分类器（使用各分类器的默认参数）

In [ ]:
g_classifiers = [
    ('Logistic Regression', LogisticRegression()),
    ('KNN', KNeighborsClassifier()),
    ('SVM', SVC()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Random Forest', RandomForestClassifier()),
    ('AdaBoost', AdaBoostClassifier()),
    ('GradientBoosting', GradientBoostingClassifier()),
    ('Bagging', BaggingClassifier()),
    ('LDA', LinearDiscriminantAnalysis()),
    ('MLP', MLPClassifier())
]

# 使用训练集进行训练
并在测试集上使用判别系数进行评估

In [ ]:
import pandas as pd
df = pd.DataFrame()
models={}
for name, clf in g_classifiers:
    print("Begin train", name)
    clf.fit(train_x,train_y)
    models[name] = clf
    score = clf.score(test_x,test_y)
    df=df.append({"Model":name,"Score":score},ignore_index=True)
#输出各分类器的评测结果
display(df)
df.to_csv("./output/result.csv")

# 使用模型进行分类，并显示结果

In [ ]:
fig,ax = plt.subplots(5,7,figsize=(6,6))
def ShowImg(id,title,img,cm = 'gray'):
    x,y = divmod(id,7)
    ax[x,y].imshow(img,plt.get_cmap(cm))
    ax[x,y].set_title(title,fontsize=16)
    ax[x,y].axis('off')
for i in range(35):
    image_hog=test_x[i]
    label = models['SVM'].predict([image_hog])
    ShowImg(i,label[0],test_image[i].reshape(28,28))